In [4]:
import numpy as np
import random
from inputdata.Course import Course
from inputdata.Room import Room
from inputdata.Student import Student
from inputdata.Teacher import Teacher
from Schedule import Schedule 
from MasterSchedule import MasterSchedule
from RoomSchedule import RoomSchedule
from StudentSchedule import StudentSchedule
from TeacherSchedule import TeacherSchedule
from Section import Section
from Constants import DAYS_IN_WEEK, NUMBER_OF_PERIODS

In [23]:
teachers = np.array([], dtype=Teacher)
students = np.array([], dtype=Student)

In [2]:
def SplitSectionPeriod(master_schedule : MasterSchedule):
    # choose random room such that it is occupied at some period
    orig_room_index = random.randrange(0, master_schedule.roomSchedules.size)
    orig_room : RoomSchedule = master_schedule.roomSchedules[orig_room_index]
    while orig_room.getOccupiedSectionTimes().size() == 0:
      orig_room_index = random.randrange(0, master_schedule.roomSchedules.size)
      orig_room = master_schedule.roomSchedules[orig_room_index]
    # choose random occupied period
    occupied = orig_room.getOccupiedSectionTimes()
    orig_period = random.choice(occupied)[1]

    # choose random room such that it is unoccupied at some period
    new_room_index = random.randrange(0, master_schedule.roomSchedules.size)
    new_room : RoomSchedule = master_schedule.roomSchedules[new_room_index]
    while new_room.getEmptySectionTimes().size() == 0:
      new_room_index = random.randrange(0, master_schedule.roomSchedules.size)
      new_room = master_schedule.roomSchedules[new_room_index]
    # choose random period that is unoccupied and the teacher is available
    unoccupied = new_room.getEmptySectionTimes()
    teacher = orig_room.getScheduledSection(0, orig_period).getTeacher()
    available_periods = np.array([])
    for time in unoccupied:
       if time[1] in teacher.getPeriodsAvailable():
          for rs in master_schedule.roomSchedules:
              if rs.sections[0][time[1]].getTeacher().getId() == teacher.getId():

  


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

In [12]:
def ChangeSectionPeriod(master_schedule : MasterSchedule):
  # choose random room
  room_index = random.randrange(0, master_schedule.roomSchedules.size)
  room : RoomSchedule = master_schedule.roomSchedules[room_index]
  # choose random period
  orig_period = random.randrange(0, NUMBER_OF_PERIODS)
  # choose another random period
  new_period
  while new_period == orig_period:
    new_period = random.randrange(0, NUMBER_OF_PERIODS)
  
  # move section from orig period to second period for all days in week
  for d in range(DAYS_IN_WEEK):
    a = room.sections[d][orig_period]
    b = room.sections[d][new_period]
    room.sections[d][orig_period] = b
    room.sections[d][new_period] = a
  master_schedule.roomSchedules[room_index] = room

  return master_schedule


In [14]:
def ChangeSectionRoom(master_schedule : MasterSchedule):
    # choose random room such that it is occupied at some period
    orig_room_index = random.randrange(0, master_schedule.roomSchedules.size)
    orig_room : RoomSchedule = master_schedule.roomSchedules[orig_room_index]
    while orig_room.getOccupiedSectionTimes().size() == 0:
      orig_room_index = random.randrange(0, master_schedule.roomSchedules.size)
      orig_room = master_schedule.roomSchedules[orig_room_index]
    # choose random occupised period
    occupied = orig_room.getOccupiedSectionTimes()
    period = random.choice(occupied)[1]
    # choose random room with that period unoccupied
    room_schedules_with_period_unoccupied = master_schedule.getIndexesOfRoomsWithPeriodUnoccuppied(period)
    new_room_index = random.choice(room_schedules_with_period_unoccupied)
    new_room = master_schedule.roomSchedules[new_room_index]
    
    # move section from orig room to new room for all days
    for d in range(DAYS_IN_WEEK):
      new_room.sections[d][period] = orig_room.sections[d][period]
      orig_room.sections[d][period] = None
    master_schedule.roomSchedules[orig_room_index] = orig_room
    master_schedule.roomSchedules[new_room_index] = new_room
    
    return master_schedule

In [8]:
def ChangeSectionTeacher(master_schedule : MasterSchedule):
    # choose random room such that it is occupied at some period
    room_index = random.randrange(0, master_schedule.roomSchedules.size)
    room : RoomSchedule = master_schedule.roomSchedules[room_index]
    while room.getOccupiedSectionTimes().size() == 0:
      room_index = random.randrange(0, master_schedule.roomSchedules.size)
      room = master_schedule.roomSchedules[room_index]
    # choose random occupied period
    occupied = room.getOccupiedSectionTimes()
    period = random.choice(occupied)[1]
    # get course
    course = room.sections[0][period].getCourse()
    
    # get new teacher that is available
    unavailable_teachers = np.ones(teachers.size) # 1 = available, 0 = unavailable
    for rs in master_schedule.roomSchedules:
      for ti in range(teachers.size()):
        # if teacher is teaching class in this period, they are unavailable
        if rs.sections[0][period].getTeacher().getId() == teachers[ti].getId():
           unavailable_teachers[ti] = 0
    
    # check if period is on teacher's available periods list
    for ti in range(teachers.size()):
        if period not in teachers[ti].getPeriodsAvailable():
           unavailable_teachers[ti] = 0

    available_teachers = np.array([]) # append teacher if they can teach the course
    for i in range(unavailable_teachers.size()):
       if unavailable_teachers[i] == 1:
          if course in teachers[i].getRankedCourses:
            available_teachers = np.append(available_teachers, teachers[i])
    
    # choose random available teacher
    new_teacher = random.choice(available_teachers)

    #change the teacher of the section for all days of the week
    for d in range(DAYS_IN_WEEK):
       room.sections[d][period].setTeacher(new_teacher)
    master_schedule.roomSchedules[room_index] = room

    return master_schedule

In [ ]:
def RemoveSection(master_schedule : MasterSchedule):
     # choose random room such that it is occupied at some period
    room_index = random.randrange(0, master_schedule.roomSchedules.size)
    room : RoomSchedule = master_schedule.roomSchedules[room_index]
    while room.getOccupiedSectionTimes().size() == 0:
      room_index = random.randrange(0, master_schedule.roomSchedules.size)
      room = master_schedule.roomSchedules[room_index]
    # choose random occupised period
    occupied = room.getOccupiedSectionTimes()
    period = random.choice(occupied)[1]
    # delete section for all days in week
    for d in range(DAYS_IN_WEEK):
       room.sections[d][period] = None
    master_schedule.roomSchedules[room_index] = room

    return master_schedule

In [7]:
a = np.array([1, 2, 3])
4 not in a

True